In [218]:
import pandas as pd 
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [219]:
#import the data
train_x = pd.read_csv("TrainingSetValues.csv")
trian_y = pd.read_csv("TrainingSetLabels.csv")
test_x = pd.read_csv("TestSetValues.csv")
data = pd.read_csv("names.csv")

In [220]:
#Imputing missing values for both train and test
train_x.fillna(-999, inplace=True)
test_x.fillna(-999,inplace=True)

In [221]:
# train_x.isnull().sum()
#convert output statu group to 1,2,3 
 
replace_map = {"non functional":3, 
              "functional needs repair":2,
              "functional":1}

trian_y['status_group code'] = trian_y['status_group'].replace(replace_map)

In [222]:
#subset data train 
names = data.name.tolist()
train_x = train_x[names]
train_x = train_x.astype({"amount_tsh": int})

train_x.dtypes
#train_x.isnull().sum()

amount_tsh                int32
funder                   object
gps_height                int64
installer                object
basin                    object
region_code               int64
district_code             int64
lga                      object
ward                     object
population                int64
public_meeting           object
scheme_management        object
scheme_name              object
permit                   object
construction_year         int64
extraction_type_class    object
management_group         object
payment                  object
quality_group            object
quantity                 object
source_type              object
source_class             object
waterpoint_type          object
dtype: object

In [223]:
#subset test data 
test_id = pd.DataFrame(test_x['id'])

test_x = test_x[names]
test_x = test_x.astype({"amount_tsh": int})

# test_x.dtypes
#train_x.isnull().sum()



In [224]:
#split data just for testing 
X_train, X_test, Y_train, Y_test = train_test_split(train_x, trian_y['status_group code'], train_size=0.7, random_state=1234)
#Identify cat features for model
categorical_features_indices = np.where(train_x.dtypes != np.float)[0]

In [225]:
# #importing library and building model

# model=CatBoostClassifier(iterations=100,
#                            depth=10,
#                            loss_function='MultiClass')
                   
# model.fit(X_train, Y_train,cat_features=categorical_features_indices,eval_set=(X_test, Y_test))
# preds_class = model.predict(X_test)

In [226]:
# confusion_matrix(preds_class,Y_test)

In [227]:
# accuracy_score(preds_class,Y_test)

In [228]:
#full model 
#importing library and building model

model_full = CatBoostClassifier(iterations=10000,
                           depth=10,
                           loss_function='MultiClass')
                   
model_full.fit(train_x, trian_y['status_group code'],cat_features=categorical_features_indices)
preds_class_full = model.predict(test_x)


Learning rate set to 0.013925
0:	learn: 1.0869504	total: 1.12s	remaining: 3h 6m 8s
1:	learn: 1.0757258	total: 2.17s	remaining: 3h 30s
2:	learn: 1.0647320	total: 3.67s	remaining: 3h 23m 40s
3:	learn: 1.0537649	total: 5.15s	remaining: 3h 34m 23s
4:	learn: 1.0433860	total: 6.32s	remaining: 3h 30m 37s
5:	learn: 1.0333106	total: 7.62s	remaining: 3h 31m 37s
6:	learn: 1.0234740	total: 8.66s	remaining: 3h 26m 3s
7:	learn: 1.0140236	total: 9.7s	remaining: 3h 21m 49s
8:	learn: 1.0049360	total: 10.5s	remaining: 3h 14m 49s
9:	learn: 0.9959993	total: 11.6s	remaining: 3h 12m 55s
10:	learn: 0.9874145	total: 12.7s	remaining: 3h 12m 37s
11:	learn: 0.9791595	total: 14.3s	remaining: 3h 18m 17s
12:	learn: 0.9710643	total: 15.4s	remaining: 3h 17m 27s
13:	learn: 0.9630509	total: 16.5s	remaining: 3h 15m 37s
14:	learn: 0.9550924	total: 17.6s	remaining: 3h 14m 58s
15:	learn: 0.9475117	total: 18.7s	remaining: 3h 14m 33s
16:	learn: 0.9401391	total: 19.8s	remaining: 3h 14m 10s
17:	learn: 0.9327312	total: 20.8s	re

KeyboardInterrupt: 

In [ ]:
#submision
preds_class_full = pd.DataFrame(preds_class_full)
submission = pd.concat([test_id, preds_class_full], axis=1)
submission.columns = ['id', 'status_group']
replace_map2 = {3:"non functional", 
              2:"functional needs repair",
              1:"functional"}

submission['status_group'] = submission['status_group'].replace(replace_map2)
submission.to_csv("submision.csv", index=False)